# Phase 4 Time Series Project

## Group Members
- William Itotia
- Firdosa Mohammed
- Frank Oyugi
- Esther Terry Munene

# HOUSING YOU TOO LTD TOP 5 BEST ZIP CODES

## Problem Statement

Housing You Too Ltd, a consultancy and real estate firm, has been approached by a client eager to invest in properties with the goal of achieving the highest possible Return on Investment (ROI). The client is looking for our expert guidance to identify the top 5 zip codes that promise the best ROI. Recognizing the complexity and layered nature of this question, we are committed to providing a thorough and data-driven analysis. To achieve this, we will utilize the extensive Zillow dataset, which contains historical real estate data. By analyzing trends, property values, and other key metrics, we aim to deliver the most accurate and insightful recommendations to guide our client's investment decisions for maximum profitability.

## Objectives

1. Identify the 5 best zip codes that will bring our clients the highest ROI
